In [1]:
# Import all library 
import pandas as pd 
import pyodbc as db
import re

In [2]:
# # Read full name files 
# data = pd.read_excel('./Data/fullnamelist.xlsx')
# data.head()

In [50]:
# server connections  
connection = db.connect('DRIVER={SQL Server};'
                            'SERVER=137.116.139.217;'
                            'DATABASE=ARCHND;'
                            'UID=sa;PWD=erp@123')

query = """SELECT  CustomerInformation.IDCUST,CustomerInformation.NAMECUST,
        CustomerInformation.CUSTYPEDESC, customerinformation.AUDTORG
        from ARCHIVESKF.dbo.CustomerInformation LEFT OUTER JOIN
        ARCHIVESKF.dbo.Customer_ShortName ON Rtrim(ltrim(CustomerInformation.IDCUST)) =
        Rtrim(ltrim(Customer_ShortName.IDCUST))
        AND CustomerInformation.AUDTORG = Customer_ShortName.AUDTORG
        where Customer_ShortName.IDCUST is null
         """

In [51]:
# Fetch data 
data = pd.read_sql_query(query, connection)

In [52]:
data.head(3)

,IDCUST,NAMECUST,CUSTYPEDESC,AUDTORG
0,90000042282,Rosin Medical Hall ...,Doctor-Cum Chemist ...,HZJSKF
1,90000042314,Kholifa Pharmacy ...,Retailer-Ex. HQ ...,KHLSKF
2,90000042333,Halder Pharmacy ...,Retailer-Ex. HQ ...,RAJSKF


In [43]:
data.shape

(324, 4)

In [44]:
# Read nameing patterns file 
patterns = pd.read_excel('./Data/skf_name_patterns.xlsx')
patterns.head(3)

,actual,short
0,Association,ASSOC
1,Aushadhalay,AUSHAD
2,Aushad Ghar,AUSHAD


In [45]:
# Convert patterns dataframe to dictionary 
key = patterns.set_index('actual')['short'].to_dict()

In [46]:
# Create a new dataframe and store processed data in it 
df = pd.DataFrame()
df['short_name'] = data['NAMECUST'].replace(key, regex=True)

In [62]:
# Create a new dataframe for both Full Name and Short Name 
newdf = pd.DataFrame()
newdf['IDCUST'] = data.IDCUST.astype(str).str.strip()
newdf['NAMECUST'] = data.NAMECUST.str.strip()
newdf['Short Name'] = df.short_name.str.strip()
newdf['CUSTYPEDESC'] = data.CUSTYPEDESC.str.strip()
newdf['AUDTORG'] = data.AUDTORG.str.strip()

In [63]:
type(newdf['IDCUST'][0])

str

In [64]:
# Finally store all actual and processed data in a file 
newdf.to_csv('./Data/skf_short_name.csv', index=False)
print('short name saved')

short name saved


# Color not processed rows 

In [65]:
# Color functions 
def color_negative_red(val):
    color = 'red' if len(val) > 20 else 'black'
    return 'color: %s' % color

a = newdf.style.applymap(color_negative_red)

In [66]:
# Save output
a.to_excel('./Data/output.xlsx', index=False)
print('Wrong Output maked as red color')

Wrong Output maked as red color
